In [187]:
!pip install seaborn

## State of the art (ROAD, sort base ranking and linear noisy imputation notrain method)

In [272]:
noretrain_linear = "/workspaces/outputs/road/sota/results/noretrain.json"

noretrain_telea = "/workspaces/outputs/road/sort_telea/results/noretrain.json"
noretrain_ns = "/workspaces/outputs/road/sort_ns/results/noretrain.json"

In [273]:
import json
import pandas as pd
import numpy as np

In [274]:
with open(noretrain_linear) as json_file:
    noretrain_linear = json.load(json_file)


In [275]:
with open(noretrain_telea) as json_file:
    noretrain_telea = json.load(json_file)



In [277]:
def averaging_accuracy(dict_result):
    import copy
    
    dictionary = copy.deepcopy(dict_result)
    for method in dictionary['imputations']:
        for key_m, value_m in dictionary[method].items():
            for key_sb, value_ac in value_m.items():
                for key_f, value_f in  value_ac.items():
                    for key_p, value_ar in value_f.items():
                        value_f[key_p] = sum(value_ar)/len(value_ar)
    return dictionary

In [286]:
#averaged_accuarcy_linear = averaging_accuracy(noretrain_linear)
averaged_accuarcy_telea = averaging_accuracy(noretrain_telea)
averaged_accuracy_ns =  averaging_accuracy(noretrain_ns)

In [301]:
def plot_consistancy( data_dict, appraoch="linear", morf=True, path=None):
    import seaborn as sns
    import copy
    if morf:
        method = 'MoRF'
    else:
        method ='LeRF'
    printable_data_dict = copy.deepcopy(data_dict)
    new_dict = {}
    new_dict['x_axis'] = printable_data_dict["percentages"]
    for key_m,  val_m in printable_data_dict[appraoch].items():
        for key_var, val_var in val_m.items():
            if not morf:
                accs = val_var['lerf']
                l = list(accs.values())
                l.reverse()
                new_dict[f"{key_m}-{key_var}"] = l
            else:
                accs = val_var['morf']
                new_dict[f"{key_m}-{key_var}"] = list(accs.values())
                 

    import seaborn as sns
    sns.set_theme(style="whitegrid")

    data = pd.DataFrame.from_dict(new_dict)
    data = data.set_index('x_axis')

    ax = sns.lineplot(data=data, palette="tab10", linewidth=2.5)
    ax.set(ylim=(0.0, 1.0))
    ax.set(xlim=(0.1, 0.9))
    ax.set(xlabel=f'% romoved {method}', ylabel='Accuracy')
    ax.figure.savefig(f'{path}/{appraoch}-{method}.png')
    ax.figure.clf()
    

from scipy.stats import spearmanr
def calculate_sperman_rank(accuracy_result, method = "telea", morf=True):
    ranking = {}
    for key_m,  val_m in accuracy_result[method].items():
        for key_var, val_var in val_m.items():
            if morf:
                res = np.array(list(val_var["morf"].values()))
                per = list(val_var["morf"].keys())
                per = 1 - np.array([float(x) for x in per])
            else:
                res =  list(val_var["lerf"].values())
                res.reverse()
                res = 1 - np.array(res)
                per = list(val_var["lerf"].keys())
                
                per = [float(x) for x in per]
                
            score = spearmanr(per, res).correlation
            ranking[f"{key_m}-{key_var}"] = score
    ranking = {k: v for k, v in sorted(ranking.items(), key=lambda item: item[1])}
    
    return ranking



In [309]:
def calculate_spearman_morf_lerf(accuracy_result, method = "telea"):
    scores =[]
    for key_m,  val_m in accuracy_result[method].items():
        for key_var, val_var in val_m.items():
                morf = list(val_var["morf"].values())
                lerf =  list(val_var["lerf"].values())
                lerf.reverse()
                score = spearmanr(morf, lerf).correlation
                print("morf", morf)
                print("lerf", lerf)
                scores.append(score)
    return scores


In [303]:
#plot_consistancy(averaged_accuracy_ns, morf=False, appraoch="ns", path=f'/workspaces/outputs/road/sota/plots')

plot_consistancy(averaged_accuracy_ns, morf=True, appraoch="ns", path=f'/workspaces/outputs/road/sort_ns/plots')

<Figure size 640x480 with 0 Axes>

## Sort telea


In [304]:
plot_consistancy(averaged_accuarcy_telea, appraoch='telea', morf=True, path=f'/workspaces/outputs/road/sort_telea/plots')

plot_consistancy(averaged_accuarcy_telea, appraoch='telea', morf=True, path=f'/workspaces/outputs/road/sort_telea/plots')

<Figure size 640x480 with 0 Axes>

## Spearman correlation

In [305]:
ranking = calculate_sperman_rank(averaged_accuarcy_telea, method='telea', morf=False)
print(ranking)
ranking = calculate_sperman_rank(averaged_accuracy_ns, method='ns', morf=True)
print(ranking)

{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}
{'ig-base': 1.0, 'ig-sg': 1.0, 'ig-sq': 1.0, 'ig-var': 1.0}


In [310]:
scores = calculate_spearman_morf_lerf(averaged_accuarcy_telea)
print(scores)

morf [0.6891600131988526, 0.618779993057251, 0.5682000041007995, 0.5281599998474121, 0.49991999864578246, 0.43948000073432925, 0.31716000437736513]
lerf [0.8167399883270263, 0.814900004863739, 0.7916000127792359, 0.7779600024223328, 0.7434399962425232, 0.6636199951171875, 0.4818999946117401]
morf [0.6395000100135804, 0.5505000114440918, 0.4944200038909912, 0.454940003156662, 0.4256600081920624, 0.3770400047302246, 0.27700000405311587]
lerf [0.8229199886322022, 0.8170599937438965, 0.8049599885940552, 0.7942999958992004, 0.7700399994850159, 0.7049200057983398, 0.5195199966430664]
morf [0.7293400049209595, 0.6584599852561951, 0.578980004787445, 0.48871999979019165, 0.4073199927806854, 0.285479998588562, 0.2092600017786026]
lerf [0.790500009059906, 0.7714399933815003, 0.7256199955940247, 0.6800600051879883, 0.6126999974250793, 0.49727999567985537, 0.32896000146865845]
morf [0.732099997997284, 0.6488399982452393, 0.5609600067138671, 0.47360000014305115, 0.38816000819206237, 0.26870000362396